In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on {device}")

Running on cuda


In [2]:
def data_loader(data, batch_size, random_seed=123, valid_size=0.1, shuffle=True,test=False):
    normalise = transforms.Normalize(mean=[0.4914,0.4822,0.4465],std=[0.2023,0.1994,0.2010])
    transform = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),normalise])
    transform_train = transforms.Compose(
        [transforms.Resize((256,256)),
         transforms.RandomHorizontalFlip(),
         transforms.RandomVerticalFlip(p=0.5),
         transforms.CenterCrop(224),
         transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.4, hue=0.1),
         transforms.RandomRotation(10),
         transforms.ToTensor(),
         normalise])
    if test:
        dataset = datasets.Flowers102(root=data, split="test", download=True, transform=transform)
        data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle)
        return data_loader
    train_dataset = datasets.Flowers102(root=data, split="train", download=True, transform=transform_train)
    validation_dataset = datasets.Flowers102(root=data, split="val", download=True, transform=transform)
    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_idx)
    valid_sampler = torch.utils.data.sampler.SubsetRandomSampler(valid_idx)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, sampler=train_sampler)
    valid_loader = DataLoader(dataset=validation_dataset, batch_size=batch_size, sampler=valid_sampler)
    return (train_loader, valid_loader)

train_loader, valid_loader = data_loader(data='flowers', batch_size=32, shuffle=True, test=False)
test_loader = data_loader(data='flowers', batch_size=32, shuffle=True, test=True)

In [3]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(Block, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels)
        )
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [4]:
class Rn(nn.Module):
    def __init__(self, block, layers, num_classes=102):
        super(Rn, self).__init__()
        self.in_planes = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512, num_classes)
    
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_planes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels=self.in_planes, out_channels=planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )
        layers = []
        layers.append(block(self.in_planes, planes, stride, downsample))
        self.in_planes = planes
        for i in range(1, blocks):
            layers.append(block(self.in_planes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        return x
    

In [5]:
num_classes = 102
num_epochs = 100
learning_rate = 0.001
weight_decay = 1e-4
model = Rn(Block, [3,4,6,3], num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=weight_decay)
total_step = len(train_loader)

In [6]:
import gc
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
        gc.collect()
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted==labels).sum().item()
        print('Validation Accuracy: {} %'.format(100 * correct / total))

Epoch [1/100], Loss: 4.5137
Validation Accuracy: 1.9607843137254901 %
Epoch [2/100], Loss: 4.3637
Validation Accuracy: 3.9215686274509802 %
Epoch [3/100], Loss: 3.8531
Validation Accuracy: 5.882352941176471 %
Epoch [4/100], Loss: 3.3507
Validation Accuracy: 5.882352941176471 %
Epoch [5/100], Loss: 3.6443
Validation Accuracy: 12.745098039215685 %
Epoch [6/100], Loss: 3.4282
Validation Accuracy: 5.882352941176471 %
Epoch [7/100], Loss: 3.4025
Validation Accuracy: 12.745098039215685 %
Epoch [8/100], Loss: 2.9744
Validation Accuracy: 11.764705882352942 %
Epoch [9/100], Loss: 3.0825
Validation Accuracy: 20.58823529411765 %
Epoch [10/100], Loss: 2.8527
Validation Accuracy: 11.764705882352942 %
Epoch [11/100], Loss: 2.7788
Validation Accuracy: 12.745098039215685 %
Epoch [12/100], Loss: 3.1628
Validation Accuracy: 21.568627450980394 %
Epoch [13/100], Loss: 1.9287
Validation Accuracy: 19.607843137254903 %
Epoch [14/100], Loss: 2.2170
Validation Accuracy: 20.58823529411765 %
Epoch [15/100], Loss

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(total, 100 * correct / total))   


Accuracy of the network on the 10000 test images: 36.575052854122625 %


In [ ]:
def count_parameters(model)->int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))

21340774
